In [7]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from string import punctuation

In [8]:
lines = sc.textFile('./shakespeare.txt')

In [9]:
words = lines.flatMap(lambda line: line.split(' ')) \
                .map(lambda word: word.lower()) \
                .filter(lambda word: len(word) > 0)

In [10]:
def strip_punctuation(string):
    return ''.join(char for char in string if char not in punctuation)

stop_words = ["i", "you", "he", "she", "it", "they", "we", "us", "them", "him", "her", "my", "your", "our", "his", "its", "their", "mine", "ours", "yours", "theirs", "hers", "me", "ii","iii","iv","vi","vii","viii","ix","x","xi","xii","xiii","xiv","xv","xvi","xvii","xvii","xviii","xix","xx", "one", "two", "thee", "four", "five", "six", "seven", "eight", "nine", "ten", "will", "thy", "a","able","about","above","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","after","afterwards","again","against","ah","all","almost","alone","along","already","also","although","always","am","among","amongst","an","and","announce","another","any","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","are","aren","arent","arise","around","as","aside","ask","asking","at","auth","available","away","awfully","b","back","be","became","because","become","becomes","becoming","been","before","beforehand","begin","beginning","beginnings","begins","behind","being","believe","below","beside","besides","between","beyond","biol","both","brief","briefly","but","by","c","ca","came","can","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","could","couldnt","d","date","did","didn't","different","do","does","doesn't","doing","done","don't","down","downwards","due","during","e","each","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","et-al","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","few","ff","fifth","first","five","fix","followed","following","follows","for","former","formerly","forth","found","four","from","further","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","had","happens","hardly","has","hasn't","have","haven't","having","he","hed","hence","her","here","hereafter","hereby","herein","heres","hereupon","hers","herself","hes","hi","hid","him","himself","his","hither","home","how","howbeit","however","hundred","i","id","ie","if","i'll","im","immediate","immediately","importance","important","in","inc","indeed","index","information","instead","into","invention","inward","is","isn't","it","itd","it'll","its","itself","i've","j","just","k","keep	keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","m","made","mainly","make","makes","many","may","maybe","me","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","more","moreover","most","mostly","mr","mrs","much","mug","must","my","myself","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","no","nobody","non","none","nonetheless","noone","nor","normally","nos","not","noted","nothing","now","nowhere","o","obtain","obtained","obviously","of","off","often","oh","ok","okay","old","omitted","on","once","one","ones","only","onto","or","ord","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","owing","own","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","re","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","s","said","same","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","she","shed","she'll","shes","should","shouldn't","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","so","some","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","such","sufficiently","suggest","sup","sure	t","take","taken","taking","tell","tends","th","than","thank","thanks","thanx","that","that'll","thats","that've","the","their","theirs","them","themselves","then","thence","there","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","these","they","theyd","they'll","theyre","they've","think","this","those","thou","though","thoughh","thousand","throug","through","throughout","thru","thus","til","tip","to","together","too","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","under","unfortunately","unless","unlike","unlikely","until","unto","up","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","very","via","viz","vol","vols","vs","w","want","wants","was","wasnt","way","we","wed","welcome","we'll","went","were","werent","we've","what","whatever","what'll","whats","when","whence","whenever","where","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","which","while","whim","whither","who","whod","whoever","whole","who'll","whom","whomever","whos","whose","why","widely","willing","wish","with","within","without","wont","words","world","would","wouldnt","www","x","y","yes","yet","you","youd","you'll","your","youre","yours","yourself","yourselves","you've","z","zero"]

words = words.map(lambda word: strip_punctuation(word).strip()) \
             .filter(lambda word: not (word in stop_words or word.isdigit())) \
             .cache()

In [11]:
WordSchema = StructType([StructField('word',  StringType(),  False), StructField('times', IntegerType(), False)])

words_frequency = sqlCtx.createDataFrame(words.map(lambda word: (word, 1)), WordSchema) \
                        .groupBy('word') \
                        .sum('times') \
                        .withColumnRenamed('sum(times)', 'times') \
                        .orderBy(desc('times')) \
                        .cache()

In [12]:
words_frequency.show(24)

+------+-----+
|  word|times|
+------+-----+
|  lord| 3059|
|  king| 2861|
|  good| 2812|
|   sir| 2754|
|  well| 2462|
| enter| 2098|
|  love| 2053|
|   ill| 1972|
|  hath| 1941|
|   man| 1835|
|   tis| 1405|
|  time| 1265|
| speak| 1165|
|  duke| 1071|
|exeunt| 1035|
| heart|  985|
|  exit|  984|
| queen|  943|
|  doth|  939|
|   art|  915|
| great|  899|
|  hear|  871|
|  lady|  864|
| death|  858|
+------+-----+
only showing top 24 rows



**Death** is the 24th most used word in Shakespeare's writings. 